# 深刻理解Python中的元类(metaclass)
http://python.jobbole.com/21351/

In [8]:
type.__subclasses__

<method '__subclasses__' of 'type' objects>

## Stackoverflow: What's metaclass
http://stackoverflow.com/questions/100003/what-is-a-metaclass-in-python

In [10]:
#python 2.7
def make_hook(f):
    """Decorator to turn 'foo' method into '__foo__'"""
    f.is_hook = 1
    return f

class MyType(type):
    def __new__(cls, name, bases, attrs):

        if name.startswith('None'):
            return None

        # Go over attributes and see if they should be renamed.
        newattrs = {}
        for attrname, attrvalue in attrs.iteritems():
            if getattr(attrvalue, 'is_hook', 0):
                newattrs['__%s__' % attrname] = attrvalue
            else:
                newattrs[attrname] = attrvalue

        return super(MyType, cls).__new__(cls, name, bases, newattrs)

    def __init__(self, name, bases, attrs):
        super(MyType, self).__init__(name, bases, attrs)

        # classregistry.register(self, self.interfaces)
        print "Would register class %s now." % self

    def __add__(self, other):
        class AutoClass(self, other):
            pass
        return AutoClass
        # Alternatively, to autogenerate the classname as well as the class:
        # return type(self.__name__ + other.__name__, (self, other), {})

    def unregister(self):
        # classregistry.unregister(self)
        print "Would unregister class %s now." % self

class MyObject:
    __metaclass__ = MyType


class NoneSample(MyObject):
    pass

# Will print "NoneType None"
print type(NoneSample), repr(NoneSample)

class Example(MyObject):
    def __init__(self, value):
        self.value = value
    @make_hook
    def add(self, other):
        return self.__class__(self.value + other.value)

# Will unregister the class
Example.unregister()

inst = Example(10)
# Will fail with an AttributeError
#inst.unregister()

print inst + inst
class Sibling(MyObject):
    pass

ExampleSibling = Example + Sibling
# ExampleSibling is now a subclass of both Example and Sibling (with no
# content of its own) although it will believe it's called 'AutoClass'
print ExampleSibling
print ExampleSibling.__mro__

Would register class <class '__main__.MyObject'> now.
<type 'NoneType'> None
Would register class <class '__main__.Example'> now.
Would unregister class <class '__main__.Example'> now.
Would register class <class '__main__.Sibling'> now.
Would register class <class '__main__.AutoClass'> now.
<class '__main__.AutoClass'>
(<class '__main__.AutoClass'>, <class '__main__.Example'>, <class '__main__.Sibling'>, <class '__main__.MyObject'>, <type 'object'>)


## Classes as objects

In [12]:
#In most languages, classes are just pieces of code that describe how to produce an object. 
class ObjectCreator(object):
    pass
my_object = ObjectCreator()

In [13]:
my_object

#### you can pass a class as a parameter

In [18]:
print(ObjectCreator)

<class '__main__.ObjectCreator'>


In [23]:
def echo(o):
    print(o)

In [17]:
echo(ObjectCreator)

<class '__main__.ObjectCreator'>


#### you can add attributes to a class

In [19]:
print(hasattr(ObjectCreator,'new_attribute'))

False


In [24]:
ObjectCreator.new_attribute = 'foo'

In [22]:
print(hasattr(ObjectCreator,'new_attribute'))

True


In [25]:
print(ObjectCreator.new_attribute)

foo


#### you can assign a class to a variable

In [26]:
ObjectCreatorMirror = ObjectCreator

In [28]:
print(ObjectCreatorMirror.new_attribute)

foo


In [29]:
print(ObjectCreatorMirror)

<class '__main__.ObjectCreator'>


## Creating classes dynamically

In [32]:
def choose_class(name):
    if name == 'foo':
        class Foo(object):
            pass
        return Foo  # return the class, not an instance
    else:
        class Bar(object):
            pass
        return Bar

In [35]:
MyClass = choose_class('foo')
print(MyClass)

<class '__main__.Foo'>


In [36]:
print(MyClass())

In [39]:
#e.g.
class MyShinyClass(object):
    pass

In [40]:
#can be created manually this way
MyShinyClass1 = type('MyShinyClass1',(),{})

In [45]:
#You'll notice that we use "MyShinyClass" as the name of the class and as the variable to hold the class reference. 
print(MyShinyClass1)

<class '__main__.MyShinyClass1'>


In [46]:
print(MyShinyClass1())

In [47]:
# type accepts a dictionary to define the attributes of the class
#>>> class Foo(object):
#...       bar = True
Foo = type('Foo', (), {'bar':True})

In [48]:
print(Foo)

<class '__main__.Foo'>


In [49]:
print(Foo.bar)

True


In [51]:
f = Foo()
print(f)

In [52]:
print(f.bar)

True


In [54]:
#you can inherit from it, so:
#>>>   class FooChild(Foo):
#...         pass
#would be:
FooChild = type('FooChild',(Foo,),{})

In [55]:
print(FooChild)

<class '__main__.FooChild'>


In [60]:
#Eventually you'll want to add methods to your class. 
#Just define a function with the proper signature and assign it as an attribute.
def echo_bar(self):
    print(self.bar)
FooChild = type('FooChild',(Foo,),{'echo_bar':echo_bar})    

In [61]:
hasattr(Foo,'echo_bar')

False

In [62]:
hasattr(FooChild,'echo_bar')

True

In [ ]:
#And you can add even more methods after you dynamically create the class, 
#just like adding methods to a normally created class object.

# What are metaclasses (finally)
type is the metaclass Python uses to create all classes behind the scenes.

It's a matter of consistency with str, the class that creates strings objects, and int the class that creates integer objects. type is just the class that creates class objects.

You see that by checking the __class__ attribute.

Everything, and I mean everything, is an object in Python. That includes ints, strings, functions and classes. All of them are objects. And all of them have been created from a class:



In [63]:
age = 35
age.__class__

int

In [64]:
name = 'Bin'
name.__class__

str

In [65]:
def foo():pass
foo.__class__

function

In [66]:
class Bar(object):pass
Bar.__class__

type

In [67]:
b = Bar()
b.__class__

__main__.Bar

In [70]:
#now, what is the __class__ of any __class__ ?
age.__class__.__class__

type

In [71]:
name.__class__.__class__

type

In [72]:
foo.__class__.__class__

type

In [73]:
b.__class__.__class__

type

# The __metaclass__ attribute

# Custom metaclasses

The main purpose of a metaclass is to change the class automatically, when it's created.

You usually do this for APIs, where you want to create classes matching the current context.



In [74]:
# the metaclass will automatically get passed the same argument
# that you usually pass to `type`
def upper_attr(future_class_name, future_class_parents, future_class_attr):
  """
    Return a class object, with the list of its attribute turned 
    into uppercase.
  """

  # pick up any attribute that doesn't start with '__' and uppercase it
  uppercase_attr = {}
  for name, val in future_class_attr.items():
      if not name.startswith('__'):
          uppercase_attr[name.upper()] = val
      else:
          uppercase_attr[name] = val

  # let `type` do the class creation
  return type(future_class_name, future_class_parents, uppercase_attr)

__metaclass__ = upper_attr # this will affect all classes in the module

class Foo(): # global __metaclass__ won't work with "object" though
  # but we can define __metaclass__ here instead to affect only this class
  # and this will work with "object" children
  bar = 'bip'

print(hasattr(Foo, 'bar'))
# Out: False
print(hasattr(Foo, 'BAR'))
# Out: True

f = Foo()
print(f.BAR)
# Out: 'bip'

False
True
bip


In [75]:
#let's do exactly the same, but using a real class for a metaclass:
# remember that `type` is actually a class like `str` and `int`
# so you can inherit from it
class UpperAttrMetaclass(type): 
    # __new__ is the method called before __init__
    # it's the method that creates the object and returns it
    # while __init__ just initializes the object passed as parameter
    # you rarely use __new__, except when you want to control how the object
    # is created.
    # here the created object is the class, and we want to customize it
    # so we override __new__
    # you can do some stuff in __init__ too if you wish
    # some advanced use involves overriding __call__ as well, but we won't
    # see this
    def __new__(upperattr_metaclass, future_class_name, 
                future_class_parents, future_class_attr):

        uppercase_attr = {}
        for name, val in future_class_attr.items():
            if not name.startswith('__'):
                uppercase_attr[name.upper()] = val
            else:
                uppercase_attr[name] = val

        return type(future_class_name, future_class_parents, uppercase_attr)

In [76]:
# We call type directly and we don't override or call the parent __new__. 
class UpperAttrMetaclass(type): 

    def __new__(upperattr_metaclass, future_class_name, 
                future_class_parents, future_class_attr):

        uppercase_attr = {}
        for name, val in future_class_attr.items():
            if not name.startswith('__'):
                uppercase_attr[name.upper()] = val
            else:
                uppercase_attr[name] = val

        # reuse the type.__new__ method
        # this is basic OOP, nothing magic in there
        # There is nothing special about it: __new__ always receives the class it's defined in, as first parameter. 
        # Just like you have self for ordinary methods which receive the instance as first parameter, 
        # or the defining class for class methods.
        return type.__new__(upperattr_metaclass, future_class_name, 
                            future_class_parents, uppercase_attr)

In [78]:
# So a real production metaclass would look like this:

class UpperAttrMetaclass(type): 

    def __new__(cls, clsname, bases, dct):

        uppercase_attr = {}
        for name, val in dct.items():
            if not name.startswith('__'):
                uppercase_attr[name.upper()] = val
            else:
                uppercase_attr[name] = val

        return type.__new__(cls, clsname, bases, uppercase_attr)

In [80]:
# We can make it even cleaner by using super, which will ease inheritance 
# (because yes, you can have metaclasses, inheriting from metaclasses, inheriting from type):

class UpperAttrMetaclass(type): 

    def __new__(cls, clsname, bases, dct):

        uppercase_attr = {}
        for name, val in dct.items():
            if not name.startswith('__'):
                uppercase_attr[name.upper()] = val
            else:
                uppercase_attr[name] = val

        return super(UpperAttrMetaclass, cls).__new__(cls, clsname, bases, uppercase_attr)

# Why would you use metaclasses classes instead of functions?
There are several reasons to do so:

* The intention is clear. When you read UpperAttrMetaclass(type), you know what's going to follow
* You can use OOP. Metaclass can inherit from metaclass, override parent methods. Metaclasses can even use metaclasses.
* You can structure your code better. You never use metaclasses for something as trivial as the above example. It's usually for something complicated. Having the ability to make several methods and group them in one class is very useful to make the code easier to read.
* You can hook on __new__, __init__ and __call__. Which will allow you to do different stuff. Even if usually you can do it all in __new__, some people are just more comfortable using __init__.
* These are called metaclasses, damn it! It must mean something!

# Why would you use metaclasses?
The main use case for a metaclass is creating an API. A typical example of this is the Django ORM.